# Roleplay actions
The purpose of this notebook is to seed the model with actions to further stimulate it to react to roleplay actions such as "meet Lorem". The actions are mixed into the model and aren't part of some kind of post processing so more is left to chance. Cause if it were guaranteed, where's the fun in that!

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy
import json
import sys
sys.path.append('..')
from config import Config

In [ ]:
d = {
    'Ry': 'remyapt',
    'Lo': 'loremapt',
    'Ip': 'loremapt',
    'Br': 'bare',
    'Em': 'emeraroom',
    'Ka': ['park3', 'park2', 'park1'],
    'Rz': ['park3', 'park2', 'park1'],
    'Kv': ['park3', 'park2', 'park1'],
    'Zh': 'bare',
    'Mv': ['park3', 'park2', 'park1'],
    'An': 'corridor',
    'Ad': 'adineapt',
    'Sb': 'office'
}
{ k: { 'scenes': v } for k, v in d.items() }

In [ ]:
interactable_characters = Config.interactable_characters
interactable_characters_inv = { v: k for k, v in interactable_characters.items() }

class ActionReaction():
    def __init__(self, actions, reactions, characters):
        self.actions = actions
        self.reactions = reactions
        self.characters = characters
        
    def to_commands(self):
        command_lines = []
        for action in self.actions:
            for reaction in self.reactions:
                for character_code, character_data in self.characters.items():
                    scenes = ['%lastscene']
                    if 'scenes' in character_data:
                        scenes = character_data['scenes']
                        if type(scenes) is not list:
                            scenes = [scenes]
                    for scene in scenes:        
                        character_name = character_code
                        if character_code in interactable_characters:
                            character_name = interactable_characters[character_code]
                        command_lines.append(f'<p><msg>c "{action.replace("%character", character_name)}"<d><scn>{scene}<msg>{character_code} "{reaction}"')
                        command_lines.append(f'<p><msg>c "{action.replace("%character", character_name).lower()}"<d><scn>{scene}<msg>{character_code} "{reaction}"')
                        command_lines.append(f'<p><msg>c "{action.lower().replace("%character", character_name).lower()}"<d><scn>{scene}<msg>{character_code} "{reaction}"')
        return command_lines

# Character actions
These actions relate to characters, each action will be paired to a character response.

In [ ]:
actions = [
    ActionReaction([
        "Meet with %character",
        "Visit %character",
        "Go to %character",
    ], [
        "Hey [player_name]!",
        "Hey [player_name]! How are you?",
        "Oh, [player_name], I didn't expect you to arrive so early.",
        "Oh, [player_name], I wasn't expecting visitors.",
    ], 
    {
        'Ry': {'scenes': 'remyapt'},
        'Lo': {'scenes': 'loremapt'},
        'Ip': {'scenes': 'loremapt'},
        'Br': {'scenes': 'bare'},
        'Em': {'scenes': 'emeraroom'},
        'Ka': {'scenes': ['park3', 'park2', 'park1']},
        'Rz': {'scenes': ['park3', 'park2', 'park1']},
        'Kv': {'scenes': ['park3', 'park2', 'park1']},
        'Zh': {'scenes': 'bare'},
        'Mv': {'scenes': ['park3', 'park2', 'park1']},
        'An': {'scenes': 'corridor'},
        'Ad': {'scenes': 'adineapt'},
        'Sb': {'scenes': 'office'}
    }),
    ActionReaction([
        "Fight %lastcharacter",
        "Attack %lastcharacter"
    ], [
        "%lastcharacter barely avoids my attack and fell, but managed to get up and quickly punch me in the face, a soaring pain quickly came over my face",
        "%lastcharacter dodges my attack and comes rushing towards me",
        "I didn't hesitate and kicked %lastcharacter right in the stomach"
    ], 
    {
        'm': {'scenes': '%lastscene'},
    }),
    ActionReaction([
        "Hey %character! Over here!",
        "Hello %character",
        "%character? Is that you?",
    ], [
        "Hey [player_name]!",
        "Hey [player_name]! How are you?",
        "[player_name]? What are you doing here?",
    ], 
    {
        k: {'scenes': '%lastscene'} for k, v in interactable_characters.items()
    })
]

command_lines = []
for action in actions:
    command_lines += action.to_commands()
print("\n".join(command_lines))